In [1]:
%pip install beautifulsoup4 requests pandas -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
BASE_URL = 'https://myanimelist.net/topanime.php?limit='
res = requests.get(F'{BASE_URL}0')
soup = bs(res.text, 'html.parser')

In [4]:
animes = []
def get_anime_values(tr):
    tds = tr.find_all('td')
    position = int(tds[0].text.strip()) if '-' not in tds[0].text.strip() else np.nan
    title = tds[1].find_all('h3')[0].text.strip()
    info = tds[1].find('div', {'class': 'information di-ib mt4'}).text.strip().split('\n')
    info = [i.strip() for i in info]
    episodes = info[0]
    release_date = info[1]
    members = int(info[2].replace(',', '').split()[0])
    score = float(tds[2].text.strip()) if 'N/A' not in tds[2].text else 0
    details = tr.find_all('a')[0].get('href').strip()
    
    return [position, title, episodes, release_date, members, score, details]

In [5]:
%%time
RANGE_LIMIT = 100000 # 100000 is the max limit
animes = []
for i in range(0, RANGE_LIMIT, 50):
    res = requests.get(F'{BASE_URL}{i}')
    soup = bs(res.text, 'html.parser')
    if r"This page doesn't exist." in soup.text:
        print('Finished')
        break
    print(f'Getting data from {i} to {i+50}')
    animes.extend(get_anime_values(tr) for tr in soup.find_all('tr')[1:])

Getting data from 0 to 50
Getting data from 50 to 100
CPU times: total: 469 ms
Wall time: 2.63 s


In [6]:
df = pd.DataFrame(animes, columns=['position', 'title', 'episodes', 'release_date', 'members', 'score', 'details'])
df.drop_duplicates(subset=['title', 'episodes', 'release_date'], inplace=True, ignore_index=True)
df.sample(5)

,position,title,episodes,release_date,members,score,details
92,93,Kimi no Suizou wo Tabetai,Movie (1 eps),Sep 2018 - Sep 2018,822148,8.56,https://myanimelist.net/anime/36098/Kimi_no_Su...
15,16,Clannad: After Story,TV (24 eps),Oct 2008 - Mar 2009,1110529,8.94,https://myanimelist.net/anime/4181/Clannad__Af...
79,80,Mushishi Zoku Shou: Suzu no Shizuku,Movie (1 eps),May 2015 - May 2015,118617,8.60,https://myanimelist.net/anime/28957/Mushishi_Z...
65,66,Jujutsu Kaisen,TV (24 eps),Oct 2020 - Mar 2021,2034749,8.66,https://myanimelist.net/anime/40748/Jujutsu_Ka...
63,64,Hajime no Ippo: New Challenger,TV (26 eps),Jan 2009 - Jul 2009,252361,8.66,https://myanimelist.net/anime/5258/Hajime_no_I...


In [7]:
df[df.title.str.contains(',')].shape

(0, 7)

In [8]:
df[df.title.str.contains(';')].shape

(1, 7)

In [9]:
df.title = df.title.str.replace(',', '')

In [10]:
%%time
df.to_csv('AllAnimes.csv', index=False)

CPU times: total: 15.6 ms
Wall time: 16 ms
